In [1]:
import os
import json
import ray
import numpy as np
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
from flow.networks import Network
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams, VehicleParams, SumoCarFollowingParams, InFlows, SumoLaneChangeParams
from flow.controllers import RLController, IDMController, SimLaneChangeController
from gym.spaces.box import Box
from gym.spaces import Tuple

from constructionRouter import ConstructionRouter
from constructionEnv_simplified import myEnv

# time horizon of a single rollout
HORIZON = 5000 #5000
# number of rollouts per training iteration
N_ROLLOUTS = 10
# number of parallel workers
N_CPUS = 1

vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(RLController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             routing_controller=(ConstructionRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
             ),
             num_vehicles=8
             )
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode=25
             ),
             lane_change_params = SumoLaneChangeParams(lane_change_mode=1621),
             num_vehicles=0)

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["edge4"]
)

# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {
                "edge1": ["edge1","edge2","edge3","edge4","edge5","edge6"],
                #"edge3": ["edge3","edge4","edge5","edge10","edge11","edge12","edge3"],
                "edge4": ["edge4","edge5","edge10","edge11","edge12","edge3","edge4"],
                }


inflow = InFlows()
inflow.add(veh_type="human",
           edge="edge1",
           vehs_per_hour=3000,
            depart_lane="random",
            depart_speed="random",
            color="white")

file_dir = "/home/llavezzo/"
net_params = NetParams(
    template="/mnt/c/users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/construction/constructionV6.net.xml",
    inflows=inflow
)

flow_params = dict(
    # name of the experiment
    exp_tag="construction_traffic_simplified",

    # name of the flow environment the experiment is running on
    env_name=myEnv,  # <------ here we replace the environment with our new environment

    # name of the network class the experiment is running on
    network=Network,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.5,
        render=False,
        restart_instance=True,
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=1000,
        clip_actions=False,
        additional_params={
            "target_velocity": 50,
            "sort_vehicles": False,
            "max_accel": 2,
            "max_decel": 2,
            "lane_change_duration": 5,
        },
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=initial_config
)


def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"

    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [32, 32, 32, 32]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps(
        flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    return alg_run, gym_name, config




/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [ ]:
alg_run, gym_name, config = setup_exps()
ray.init(num_cpus=N_CPUS + 1)
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 5,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 200,
        },
    }
})

2020-04-10 17:47:35,284	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-04-10 17:47:35,286	INFO resource_spec.py:212 -- Starting Ray with 4.49 GiB memory available for workers and up to 2.27 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-10 17:47:35,669	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


Trial name,status,loc
PPO_myEnv-v0_e443a326,RUNNING,


(pid=raylet) E0410 17:47:35.863157500    8037 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1586555255.863130800","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=raylet) E0410 17:47:35.863447100    8037 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=raylet) E0410 17:47:35.890820900    8037 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=8058) E0410 17:47:37.258147800    8058 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1586555257.258133800","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=8

(pid=8060) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=8060)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=8060) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=8060)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=8060) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=8060)   _np_qint16 = np.dtype([("qint16", np.int16, 1)]

(pid=8060) 11.830687816537631
(pid=8060) 11.824216829524127
(pid=8060) 11.821119918252052
(pid=8060) 11.77372759730862
(pid=8060) 11.735022078712923
(pid=8060) 11.781734614242428
(pid=8060) 11.77766970240453
(pid=8060) 11.80108479715994
(pid=8060) 11.778666865212788
(pid=8060) 11.702611282891567
(pid=8060) Warning: Vehicle 'flow_00.36' performs emergency braking with decel=-23.45 wished=4.50 severity=4.21, time=92.50.
(pid=8060) Warning: Vehicle 'flow_00.34' performs emergency braking with decel=-18.70 wished=4.50 severity=3.16, time=92.50.
(pid=8060) 11.70131563345305
(pid=8060) 11.699828185714757
(pid=8060) 11.697305228163383
(pid=8060) 11.67329419170926
(pid=8060) 11.551013749408712
(pid=8060) 11.578737315599303
(pid=8060) 11.580667132536762
(pid=8060) 11.60731395379254
(pid=8060) Warning: Vehicle 'rl_5' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=96.00.
(pid=8060) 11.59804585751168
(pid=8060) 11.615728047783596
(pid=8060) 11.628322742298907
(pid=8060

...

(pid=8060) 3.6273029521931597
(pid=8060) 3.63183922693857
(pid=8060) 3.6393953092793354
(pid=8060) 3.5838404044619843
(pid=8060) 3.640292505586968
(pid=8060) 3.63398159947149
(pid=8060) 3.604704687478202
(pid=8060) 3.5466721010635247
(pid=8060) 3.6034257219413086
(pid=8060) 3.594464599809761
(pid=8060) 3.5970244796909254
(pid=8060) 3.538637858487944
(pid=8060) 3.5411888782948076
(pid=8060) 3.586224990215282
(pid=8060) 3.5878215184253315
(pid=8060) 3.5288810179287573
(pid=8060) 3.4562420180654385
(pid=8060) 3.5743142216821613
(pid=8060) 3.5746925487900576
(pid=8060) 3.5140463102612536
(pid=8060) 3.4960986483556584
(pid=8060) 3.5579732001509643
(pid=8060) 3.5554659193324025
(pid=8060) 3.4927343029565345
(pid=8060) 3.4516306146188085
(pid=8060) 3.511284183818592
(pid=8060) 3.507808563150845
(pid=8060) 3.4450694395937425
(pid=8060) 3.42403915857537
(pid=8060) 3.4799641769062495
(pid=8060) 3.4751039612101584
(pid=8060) 3.3974520403468222
(pid=8060) 3.386706255774433
(pid=8060) 3.44300110505

(pid=8060) 3.150024690818173
(pid=8060) 3.0922314244972315
(pid=8060) 3.1989253998870435
(pid=8060) 3.202576709738309
(pid=8060) 3.147539644180118
(pid=8060) 3.081140845265989
(pid=8060) 3.176462956337469
(pid=8060) 3.204079886062871
(pid=8060) 3.145588394322901
(pid=8060) 3.0738845004956596
(pid=8060) 3.1431171976819643
(pid=8060) 3.1715359526543825
(pid=8060) 3.1176736525667055
(pid=8060) 3.0617923625257815
(pid=8060) 3.1365501679320658
(pid=8060) 3.1506085151799352
(pid=8060) 3.102587552684824
(pid=8060) 3.0473657121783915
(pid=8060) 3.1303657032519863
(pid=8060) 3.149735441495442
(pid=8060) 3.102377763132368
(pid=8060) 3.1153404288632176
(pid=8060) 3.1352956916111676
(pid=8060) 3.1532531004176803
(pid=8060) 3.107488791863535
(pid=8060) 3.1211394464811684
(pid=8060) 3.137738520035504
(pid=8060) 3.156195215520659
(pid=8060) 3.11744129379223
(pid=8060) 3.130286462260018
(pid=8060) 3.1489299143431015
(pid=8060) 3.1090495351783245
(pid=8060) 3.1268825425564506
(pid=8060) 3.1366477625672

(pid=8060) 1.7443645230184068
(pid=8060) 1.6808718918924237
(pid=8060) 1.7499983867743913
(pid=8060) 1.7960483055738576
(pid=8060) 1.8327911901082354
(pid=8060) 1.8651782211782657
(pid=8060) 1.8886270646631242
(pid=8060) 1.81292019646952
(pid=8060) 1.8133334359444946
(pid=8060) 1.8188475242050772
(pid=8060) 1.8114352188814733
(pid=8060) 1.849922738331724
(pid=8060) 1.8707894066303317
(pid=8060) 1.7956597374613417
(pid=8060) 1.8147662343459507
(pid=8060) 1.8367055427081036
(pid=8060) 1.8563625269942399
(pid=8060) 1.84411464473346
(pid=8060) 1.766626086264345
(pid=8060) 1.788864871972097
(pid=8060) 1.795490449866572
(pid=8060) 1.816467210182653
(pid=8060) 1.8353156379446796
(pid=8060) 1.8746255914160348
(pid=8060) 1.9042117652033024
(pid=8060) 1.7919191621557415
(pid=8060) 1.803680690260962
(pid=8060) 1.812883596556402
(pid=8060) 1.7965274428524132
(pid=8060) 1.868106005250216
(pid=8060) 1.8714718551820995
(pid=8060) 1.9408983491125504
(pid=8060) 1.9749439032600191
(pid=8060) 1.982397532

(pid=8060) 1.555046973393343
(pid=8060) 1.5203211349326455
(pid=8060) 1.4893875445229883
(pid=8060) 1.540814562680879
(pid=8060) 1.4992597479995795
(pid=8060) 1.4784667869532886
(pid=8060) 1.5105053066201808
(pid=8060) 1.482364682533205
(pid=8060) 1.4539744026010124
(pid=8060) 1.4288012025310648
(pid=8060) 1.4575142720747138
(pid=8060) 1.426807094464536
(pid=8060) 1.3949416255100673
(pid=8060) 1.34812194303981
(pid=8060) 1.380513878700754
(pid=8060) 1.3209535795421161
(pid=8060) 1.2839760177703143
(pid=8060) 1.2357762059589232
(pid=8060) 1.2665722746334371
(pid=8060) 1.2304238718900569
(pid=8060) 1.1908195132401325
(pid=8060) 1.1441883203341023
(pid=8060) 1.1491159975001615
(pid=8060) 1.102306622430772
(pid=8060) 1.0501595385985791
(pid=8060) 1.0054090908861257
(pid=8060) 1.0275384987583165
(pid=8060) 0.9888084355864253
(pid=8060) 0.9418043410280149
(pid=8060) 0.8946382024790356
(pid=8060) 0.933046368763375
(pid=8060) 0.8933355460671091
(pid=8060) 0.8560693576527486
(pid=8060) 0.812966

(pid=8060) 0.2729239446773001
(pid=8060) 0.27016810936199803
(pid=8060) 0.2686456194474875
(pid=8060) 0.2777946367394537
(pid=8060) 0.2839798646342819
(pid=8060) 0.2849636466669909
(pid=8060) 0.28038297356667785
(pid=8060) 0.319878986200623
(pid=8060) 0.32440935026575846
(pid=8060) 0.3165994946631814
(pid=8060) 0.32286975747905067
(pid=8060) 0.32891761252377016
(pid=8060) 0.3435068291736816
(pid=8060) 0.33494968725591445
(pid=8060) 0.3430957650978396
(pid=8060) 0.3600282073076276
(pid=8060) 0.36759260236781505
(pid=8060) 0.3790983751964812
(pid=8060) 0.3938347813788622
(pid=8060) 0.38501445150144065
(pid=8060) 0.40118604904463623
(pid=8060) 0.41235120470634745
(pid=8060) 0.42123190036461794
(pid=8060) 0.42708603201642054
(pid=8060) 0.4402494448618756
(pid=8060) 0.4602268189476525
(pid=8060) 0.4706855332280898
(pid=8060) 0.48732064746653986
(pid=8060) 0.5123580210087074
(pid=8060) 0.515516766087893
(pid=8060) 0.5313631329895507
(pid=8060) 0.5430471411429053
(pid=8060) 0.5246377264879561